## Групповой проект команды FlexML. Exploratory Data Analysis
 - Гуськова Мария
 - Мусин Олег
 - Сухотин Олег
 - Ванцов Иван

## Первичный анализ полученных данных
Проведем первичную очистку данных, полученных от Netflix.

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as sps
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import plotly.express as px
warnings.filterwarnings('ignore')

In [ ]:
netflix_shows = pd.read_excel('assets/NetflixShows.xlsx')
netflix_shows.info() # для определения типа данных в колонках, анализа возможных пропусков
netflix_shows["ratingDescription"].sort_values() # для определения диапазона значений колонки
ns_cleared = netflix_shows.drop_duplicates() # для удаление дубликатов
ns_cleared = ns_cleared.drop('ratingLevel', axis=1) # удаляем колонку ratingLevel, ratingDescription, axis=1 = означает колонки (0 - строки)
ns_cleared = ns_cleared.drop('ratingDescription', axis=1)

ns_cleared.head
ns_cleared.isna().sum() # посчитаем, сколько значений nan, чтобы убедиться, что они только в колонке user rating score


# переименования для удобства
# release year -> release
# user rating score -> score
# user rating size -> size
ns_cleared = ns_cleared.rename(columns={'release year': 'release'})
ns_cleared = ns_cleared.rename(columns={'user rating score': 'score'})
ns_cleared = ns_cleared.rename(columns={'user rating size': 'size'})

ns_cleared.describe() # проанализируем данные в таблице в общем виде


1000 записей всего.
Разберем колонки нашего датасета и предложим спектр проблем или особенностей
| Колонка           | Текущий тип | Комментарий                                                                                                                               |
| ----------------- | ----------- | ----------------------------------------------------------------------------------------------------------------------------------------- |
| title             | object      | Любая последовательность букв на латинице и цифр (есть пример - "3%"). Есть риск дубликатов по неполному совпадению, требуется проверить. |
| rating            | object      | Код категории, пропусков нет                                                                                                              |
| ratingLevel       | object      | Описаниние категории, может отсутствовать. Дополнительного смысла, отличного от ragingLevel, не несет.                                    |
| ratingDescription | int64       | Закодированное значение рейтинга. Диапазон значений: от 10 до 124. Способ формирования - неизвестен                                       |
| release year      | int64       | Год выпуска, пропусков нет.                                                                                                               |
| user rating score | float64     | Числовое значение рейтинга. Может отсутствовать, вместо пропусков - NA (244 пропуска)                                                     |
| user rating size  | int64       | Общий рейтинг пользователей. Пропусков или не-числовых значений - нет.                                                                    |




EDA
 - удаление колонки ratingLevel, как не несущей дополнительного смысла
 - удаление дубликатов. В исходной таблице 500 полных дублей.
 - что у нас в таблице - фильмы и/или сериалы?
 - анализ "похожести" названий - сколько дублей по этому признаку?
 

Результаты "очищенного" датасета, первые выводы:
 - данные по 500 уникальным шоу
 - данные по шоу с 1940 по 2017 год. Больше всего данных по шоу с 2014 года

Далее повышаем детализацию первичного анализа и ищем более интересные признаки.


In [ ]:
# построим гистограмму распределения user rating score
plt.hist(ns_cleared['score'], bins = 100, edgecolor = 'r')
plt.xlabel('Score')
plt.ylabel('Number of shows')
plt.title('Shows Score Distribution')



На основании гистограммы, видно что кол-во шоу с рейтингом выше 80 больше, чем обратных

In [ ]:
# построим гистограмму распределения user rating size
plt.hist(ns_cleared['size'], bins = 100, edgecolor = 'r')
plt.xlabel('Size')
plt.ylabel('Number of score size')
plt.title('Shows Score Size Distribution')

На основании гистограммы, заметим что существует всего 3 типа score size (80, 81, 82). Что это за данные - пока не понятно.

In [ ]:
# Посмотрим, как оценки на фильмы менялись во времени
plt.figure(figsize=(12, 6))
sns.lineplot(x='release', y='score', data=ns_cleared, marker='o', color='r')
plt.title('Score Change Over Release year', fontsize=16)
plt.xlabel('Release', fontsize=12)
plt.ylabel('Score', fontsize=12)
plt.show()

In [53]:
# Посмотрим, как категории фильмов менялись во времени. Посчитаем количества всех категорий по годам.

count       500
unique       13
top       TV-14
freq        106
Name: rating, dtype: object

Проанализируем возрастные категории шоу.
Уникальных категорий: 13.
Самая популярная: TV-14.

Проанализируем кол-во шоу в разных категориях

In [ ]:
#ns_cleared["rating"].describe()
# возьмем значения с 1970 года, чтобы получить более наглядные данные (убираем только 1 фильм 1940 года)
category_distribution = ns_cleared[ns_cleared["release"] > 1970].groupby(by=["release", "rating"]).size().reset_index(name='count')
category_distribution.describe

fig = px.histogram(category_distribution, x='release',
    y='count',
    color='rating',
    barmode='group',
    title='Распределение групп шоу по годам',
    text_auto=True)

# Adjust the layout for visibility
fig.update_layout(xaxis=dict(title='Release Year'),
    yaxis=dict(title='Number of Shows'),
    legend_title="Rating",
    bargap=0.2)
fig.show()

In [ ]:
# Посмотрим на корреляцию оценок и рейтинга фильма